# **Notas**
## **Alunos:**
    **Leydson Barros**    
    **Leandro Damaceno**


---



> https://medium.com/data-hackers/uma-introdu%C3%A7%C3%A3o-simples-ao-pandas-1e15eea37fa1



# --**Brazilian Cities**--


1.   Classificação: Ouro
2.   Usabilidade: 10





> https://www.kaggle.com/datasets/crisparada/brazilian-cities


# --**Importações bibliotecas**-- 

In [ ]:
import numpy as np
import pandas as pd
from numpy import log10
import matplotlib.pyplot as plt
%matplotlib inline


# **Base de dados** 

In [ ]:

df_brazil=pd.read_csv("datas\BRAZIL_CITIES_REV2022.CSV",sep=",")
df_dictionary=pd.read_csv("datas\Data_Dictionary.csv",sep=";")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


# ***Especificação***
#### 1. **Coletar	dados ok**
#### 2.	**Realizar pré-processamento	dos	dados**	


*   Definição	de	tipos - ok
*   Tratamento	de	dados	ausentes - ok
*   Normalização	e	discretização 
*   Limpeza	de	dados	(univariado,	bivariado	e	
multivariado)
#### 3.	**Apresentar estatísticas descritivas	dos	dados com	visualizações** 
#### 4.	**Realizar teste	de	hipóteses para comparação de	valores	de	categorias	(ex.	preço	m2 em diferentes bairros)	e	visualizar diferenças**


# **Bônus:**


*   Integrar	dados	ou extrair	dados	da	Web
*   Utilizar técnicas não cobertas nas aula


# **Pré-processamento** 

### Visualização dos dados

In [ ]:
# Dicionário do dataframe
df_dictionary

In [ ]:
df_brazil.head()

In [ ]:
df_brazil[['CITY','STATE','REGIAO_TUR','CATEGORIA_TUR','GVA_MAIN','RURAL_URBAN']].head(10)

In [ ]:
 #Verificando se existe dados nulos
print(df_brazil.isnull().sum())

In [ ]:
df_brazil.info()

<h1>Limpeza dos dados (Leandro)<h1> 

<h2>Verificando se existe dados duplicados<h2>

In [ ]:
len(df_brazil)


In [ ]:
#  Eliminando linhas duplicadas, Cidades: Assis, Paraty e Nova Fátima
df_brazil = df_brazil.drop_duplicates()

In [ ]:
len(df_brazil)

In [ ]:
df_brazil[df_brazil['CITY'] == 'Paraty'][['CITY','STATE','LAT','LONG']]

In [ ]:
# Cidade de Paraty possui dois dados sobre a agência de correios, eliminei o diferente de 1 (pesquisado no google que só existe uma agência)
df_brazil.drop(df_brazil.loc[(df_brazil['CITY']=='Paraty') & (df_brazil['POST_OFFICES'] != 1)].index, inplace=True)

In [ ]:
len(df_brazil)

<h2>Limpeza dos dados discrepantes<h2>

In [ ]:
# De acordo com o IBGE a menor cidade do Brasil tem 3565 km², dando o código abaixo , retona 3 cidades com área 0
df_brazil[(df_brazil['AREA'] < 3.565)][['CITY', 'STATE', 'AREA', 'IDHM']]


In [ ]:
#Limpando as cidades que tem área menor que 3.565 km², pois foipesquisado que nao existe cidade menor que essetamanho de área
df_brazil.drop(df_brazil.loc[(df_brazil['AREA'] < 3.565)].index, inplace=True)

In [ ]:
len(df_brazil)

In [ ]:
# De acordo com o IBGE a  cidade do Brasil menos povoada tem 776 pessoas
df_brazil[(df_brazil['IBGE_RES_POP'] < 776)][['CITY', 'STATE', 'AREA', 'IBGE_RES_POP']]

In [ ]:
#Eliminando as cidades que tem menos de 776 habitantes
df_brazil.drop(df_brazil.loc[(df_brazil['IBGE_RES_POP'] < 776)].index, inplace=True)

In [ ]:
len(df_brazil)

In [ ]:
#Verificando as cidades que tem IDHM menor que 0.0
df_brazil.loc[(df_brazil['IDHM'] == 0.0)][['CITY','STATE','IDHM','IDHM_Renda','IDHM_Longevidade','IDHM_Educacao']]

In [ ]:
#Eliminando as cidades que tem IDHM menor que 0.0
df_brazil.drop(df_brazil.loc[(df_brazil['IDHM'] == 0.0)].index, inplace=True)

<h2>Limpeza dos outliers usando z-score robusto (univariado)<h2>

In [ ]:
# decidi pegar a coluna GDP, o produto interno bruto de cada cidade brasileira
df_brazil['GDP'].value_counts()

In [ ]:
df_brazil['GDP'].describe()

In [ ]:
# Usando plotbox para achar os outliers
df_brazil['GDP'].plot.box()

In [ ]:
df_brazil['GDP'].hist()

In [ ]:
# Como visto em cima , os dados não estão simétricos , então aplicamos log10 e criamos uma coluna que é justamente a coluna GDP na base 10
df_brazil['GDP_Log'] = log10(df_brazil['GDP'])
df_brazil['GDP_Log'].hist()

In [ ]:
# agora podemos usar o  z-core robusto
from numpy import abs
mad = abs(df_brazil['GDP_Log'] - df_brazil['GDP_Log'].median()).median()*(1/0.6745)
print(mad)

In [ ]:
# Agora vamos plotar o z-score robusto
(abs(df_brazil['GDP_Log']-df_brazil['GDP_Log'].median())/mad).hist()

In [ ]:
# Usando a limiar > 3.25, encontro os outliers
df_brazil[abs(df_brazil['GDP_Log']-df_brazil['GDP_Log'].median())/mad > 3.25]

In [ ]:
# Gerando novo dataframe, eliminando os outros
df_brazil = df_brazil[abs(df_brazil['GDP_Log']-df_brazil['GDP_Log'].median())/mad < 3.25]

<h2>Bivariada<h2>

In [ ]:
# usando duas colunas 'COMP_TOTAL' E 'AREA', que são o total de companias de cada estado e a área de em km² respectivamente, percebesse
df_brazil.plot.scatter(x='COMP_TOT',y='AREA', figsize=(15,15), linewidths = 10)

In [ ]:
df_brazil['COMP_AREA'] = df_brazil['COMP_TOT'] / df_brazil['AREA']

In [ ]:
# Agora que fizemos a divisão, transfomando em uma univarada partir daqui repetimos na coluna comp_area o mesmo processo de z-score robusto
df_brazil['COMP_AREA'].describe()

In [ ]:
df_brazil['COMP_AREA'].plot.box()

In [ ]:
df_brazil['COMP_AREA'].hist()

In [ ]:
df_brazil['COMP_AREA_L'] = log10(df_brazil['COMP_AREA'])
df_brazil['COMP_AREA_L'].hist()

In [ ]:
mad = abs(df_brazil['COMP_AREA_L'] - df_brazil['COMP_AREA_L'].median()).median()*(1/0.6745)

In [ ]:
(abs(df_brazil['COMP_AREA_L']-df_brazil['COMP_AREA_L'].median())/mad).hist()

In [ ]:
df_brazil[abs(df_brazil['COMP_AREA_L']-df_brazil['COMP_AREA_L'].median())/mad > 3.5]

In [ ]:
df_brazil = df_brazil[abs(df_brazil['COMP_AREA_L']-df_brazil['COMP_AREA_L'].median())/mad < 3.5]

<h2>Removendo outliers com Isolation Forests (multivariado)<h2>

In [ ]:
# Primeiro passo , discretizar os dados , deixando os dados categóricos em numéricos
df_brazil['CITY'] = df_brazil['CITY'].astype('category')
df_brazil['STATE'] = df_brazil['STATE'].astype('category')
df_brazil['REGIAO_TUR'] = df_brazil['REGIAO_TUR'].astype('category')
df_brazil['CATEGORIA_TUR'] = df_brazil['CATEGORIA_TUR'].astype('category')
df_brazil['RURAL_URBAN'] = df_brazil['RURAL_URBAN'].astype('category')
df_brazil['GVA_MAIN'] = df_brazil['GVA_MAIN'].astype('category')

In [ ]:
df_brazil['CITY'] = df_brazil['CITY'].cat.codes
df_brazil['STATE'] = df_brazil['STATE'].cat.codes
df_brazil['REGIAO_TUR'] = df_brazil['REGIAO_TUR'].cat.codes
df_brazil['CATEGORIA_TUR'] = df_brazil['CATEGORIA_TUR'].cat.codes
df_brazil['RURAL_URBAN'] = df_brazil['RURAL_URBAN'].cat.codes
df_brazil['GVA_MAIN'] = df_brazil['GVA_MAIN'].cat.codes

In [ ]:
from sklearn.ensemble import IsolationForest
rng = np.random.RandomState(42)
clf = IsolationForest(max_samples=100, random_state=rng)
clf.fit(df_brazil)

In [ ]:
scores = clf.predict(df_brazil)
scores

In [ ]:
df_brazil['outlier'] = scores
print(df_brazil.head())

In [ ]:
df_brazil[df_brazil['outlier'] == -1]


In [ ]:
#nova dataframe sem os campos com -1, ou seja sem os outliers
df_brazil = df_brazil[df_brazil['outlier'] != -1]
len(df_brazil)

### Discretização de dados 

In [ ]:
#Setando campos como categóricos
#ajustando tipos dos dados
df_brazil['CAPITAL'] = df_brazil['CAPITAL'].astype('category')
df_brazil['STATE'] = df_brazil['STATE'].astype('category')
df_brazil['CATEGORIA_TUR'] = df_brazil['CATEGORIA_TUR'].astype('category')
df_brazil['GVA_MAIN'] = df_brazil['GVA_MAIN'].astype('category')
df_brazil['RURAL_URBAN'] = df_brazil['RURAL_URBAN'].astype('category')



In [ ]:
df_brazil[['CAPITAL','STATE','CATEGORIA_TUR','GVA_MAIN','RURAL_URBAN']].dtypes

In [ ]:
df_brazil['CAPITAL'].cat.categories

In [ ]:
df_brazil['STATE'].cat.categories

In [ ]:
df_brazil['CATEGORIA_TUR'].cat.categories

In [ ]:
df_brazil['GVA_MAIN'].cat.categories

In [ ]:
df_brazil['RURAL_URBAN'].cat.categories

#### IDHM

In [ ]:
df_brazil['IDHM_dist'] = 'Muito Baixo'

In [ ]:
df_brazil.loc[(df_brazil['IDHM'] <= 0.499),'IDHM_dist'] = 'Muito Baixo'
df_brazil.loc[(df_brazil['IDHM'] >= 0.5) & (df_brazil['IDHM'] <= 0.599),'IDHM_dist'] = 'Baixo'
df_brazil.loc[(df_brazil['IDHM'] >= 0.6) & (df_brazil['IDHM'] <= 0.699),'IDHM_dist'] = 'Médio'
df_brazil.loc[(df_brazil['IDHM'] >= 0.7) & (df_brazil['IDHM'] <= 0.799),'IDHM_dist'] = 'Alto'
df_brazil.loc[(df_brazil['IDHM'] >= 0.8) & (df_brazil['IDHM'] <= 1),'IDHM_dist'] = 'Muito Alto'
df_brazil['IDHM_dist'] = df_brazil['IDHM_dist'].astype('category')

In [ ]:
df_brazil['IDHM_dist'].cat.categories

In [ ]:
df_brazil['IDHM_dist'].value_counts()

In [ ]:
df_brazil['IDHM_dist'].value_counts().plot(kind='barh') 
plt.show()

In [ ]:
df_brazil[['CITY','IDHM','IDHM_dist']].head()

#### IDHM_Renda

In [ ]:
df_brazil['IDHM_Renda_dist'] = 'Muito Baixo'

In [ ]:
df_brazil.loc[(df_brazil['IDHM_Renda'] <= 0.499),'IDHM_Renda_dist'] = 'Muito Baixo'
df_brazil.loc[(df_brazil['IDHM_Renda'] >= 0.5) & (df_brazil['IDHM_Renda'] <= 0.599),'IDHM_Renda_dist'] = 'Baixo'
df_brazil.loc[(df_brazil['IDHM_Renda'] >= 0.6) & (df_brazil['IDHM_Renda'] <= 0.699),'IDHM_Renda_dist'] = 'Médio'
df_brazil.loc[(df_brazil['IDHM_Renda'] >= 0.7) & (df_brazil['IDHM_Renda'] <= 0.799),'IDHM_Renda_dist'] = 'Alto'
df_brazil.loc[(df_brazil['IDHM_Renda'] >= 0.8) & (df_brazil['IDHM_Renda'] <= 1),'IDHM_Renda_dist'] = 'Muito Alto'
df_brazil['IDHM_Renda_dist'] = df_brazil['IDHM_Renda_dist'].astype('category')

In [ ]:
df_brazil['IDHM_Renda_dist'].cat.categories

In [ ]:
df_brazil['IDHM_Renda_dist'].value_counts()

In [ ]:
df_brazil['IDHM_Renda_dist'].value_counts().plot(kind='barh') 
plt.show()

In [ ]:
df_brazil[['CITY','IDHM_Renda','IDHM_Renda_dist']].head()

#### IDHM_Longevidade

In [ ]:
df_brazil['IDHM_Longevidade_dist'] = 'Muito Baixo'

In [ ]:
df_brazil.loc[(df_brazil['IDHM_Longevidade'] <= 0.499),'IDHM_Longevidade_dist'] = 'Muito Baixo'
df_brazil.loc[(df_brazil['IDHM_Longevidade'] >= 0.5) & (df_brazil['IDHM_Longevidade'] <= 0.599),'IDHM_Longevidade_dist'] = 'Baixo'
df_brazil.loc[(df_brazil['IDHM_Longevidade'] >= 0.6) & (df_brazil['IDHM_Longevidade'] <= 0.699),'IDHM_Longevidade_dist'] = 'Médio'
df_brazil.loc[(df_brazil['IDHM_Longevidade'] >= 0.7) & (df_brazil['IDHM_Longevidade'] <= 0.799),'IDHM_Longevidade_dist'] = 'Alto'
df_brazil.loc[(df_brazil['IDHM_Longevidade'] >= 0.8) & (df_brazil['IDHM_Longevidade'] <= 1),'IDHM_Longevidade_dist'] = 'Muito Alto'
df_brazil['IDHM_Longevidade_dist'] = df_brazil['IDHM_Longevidade_dist'].astype('category')

In [ ]:
df_brazil['IDHM_Longevidade_dist'].cat.categories

In [ ]:
df_brazil['IDHM_Longevidade_dist'].value_counts()

In [ ]:
df_brazil['IDHM_Longevidade_dist'].value_counts().plot(kind='barh') 
plt.show()

#### IDHM_Educacao

In [ ]:
df_brazil['IDHM_Educacao_dist'] = 'Muito Baixo'

In [ ]:
df_brazil.loc[(df_brazil['IDHM_Educacao'] <= 0.499),'IDHM_Educacao_dist'] = 'Muito Baixo'
df_brazil.loc[(df_brazil['IDHM_Educacao'] >= 0.5) & (df_brazil['IDHM_Educacao'] <= 0.599),'IDHM_Educacao_dist'] = 'Baixo'
df_brazil.loc[(df_brazil['IDHM_Educacao'] >= 0.6) & (df_brazil['IDHM_Educacao'] <= 0.699),'IDHM_Educacao_dist'] = 'Médio'
df_brazil.loc[(df_brazil['IDHM_Educacao'] >= 0.7) & (df_brazil['IDHM_Educacao'] <= 0.799),'IDHM_Educacao_dist'] = 'Alto'
df_brazil.loc[(df_brazil['IDHM_Educacao'] >= 0.8) & (df_brazil['IDHM_Educacao'] <= 1),'IDHM_Educacao_dist'] = 'Muito Alto'
df_brazil['IDHM_Educacao_dist'] = df_brazil['IDHM_Educacao_dist'].astype('category')

In [ ]:
df_brazil['IDHM_Educacao_dist'].cat.categories

In [ ]:
df_brazil['IDHM_Educacao_dist'].value_counts()

In [ ]:
df_brazil['IDHM_Educacao_dist'].value_counts().plot(kind='barh') 
plt.show()

### Normalização
. Ainda vou fazer

# **Análise do dataSet** 

In [ ]:
#df_brazil.loc[(df_brazil['IDHM'] ==  1) | (df_brazil['IDHM'] > 0.6)][['CITY','STATE','IDHM','IDHM_dist']]

In [ ]:
# Criando os dataframes por região

nordeste = df_brazil.loc[(df_brazil['STATE'] == 'PE') | (df_brazil['STATE'] == 'MA') | (df_brazil['STATE'] == 'RN') | (df_brazil['STATE'] == 'BA') | (df_brazil['STATE'] == 'CE') | (df_brazil['STATE'] == 'AL') | (df_brazil['STATE'] == 'PI') | (df_brazil['STATE'] == 'PB') | (df_brazil['STATE'] == 'SE')]
norte =  df_brazil.loc[(df_brazil['STATE'] == 'AC') | (df_brazil['STATE'] == 'AP') | (df_brazil['STATE'] == 'AM') | (df_brazil['STATE'] == 'PA') | (df_brazil['STATE'] == 'RO') | (df_brazil['STATE'] == 'RR') | (df_brazil['STATE'] == 'TO')]
sul = df_brazil.loc[(df_brazil['STATE'] == 'PR') | (df_brazil['STATE'] == 'RS') | (df_brazil['STATE'] == 'SC')]
centroOeste =  df_brazil.loc[(df_brazil['STATE'] == 'DF') | (df_brazil['STATE'] == 'GO') | (df_brazil['STATE'] == 'MT') | (df_brazil['STATE'] == 'MS')]
sudeste = df_brazil.loc[(df_brazil['STATE'] == 'ES') | (df_brazil['STATE'] == 'MG') | (df_brazil['STATE'] == 'SP') | (df_brazil['STATE'] == 'RJ')]
# Verificação dos estados 
checknordeste = ['PE','BA', 'MA','PI','CE','RN','PB','AL','SE']
checknorte = ['AC','AP', 'AM','PA','RO','RR','TO']
checksul = ['PR','RS', 'SC']
checkcentroOeste = ['DF','GO', 'MT','MS']
checksudeste = ['ES','MG', 'SP','RJ']
# Verificação das cidades por estado
cidadesNordeste = df_brazil["STATE"].isin(checknordeste)
cidadesNorte = df_brazil["STATE"].isin(checknorte)
cidadesSul = df_brazil["STATE"].isin(checksul)
cidadesCentroOeste = df_brazil["STATE"].isin(checkcentroOeste)
cidadesSudeste = df_brazil["STATE"].isin(checksudeste)

In [ ]:
# Mapa do Brasil 

long= df_brazil["LONG"] !=0
lat = df_brazil["LAT"] !=0 
## Montado o Mapa
plt.figure(figsize=(10,10))
plt.title(" Cidades (Latitude e Longitude)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
#plt.scatter(df_brazil[long&lat&cap]["LONG"], df_brazil[long&lat&cap]["LAT"], s=50, alpha=1, label='Capital city', c ='#000000')
plt.scatter(df_brazil[long&lat&cidadesNorte]["LONG"], df_brazil[long&lat&cidadesNorte]["LAT"], s=5, alpha=1, label='NORTE', c ='#006400')
plt.scatter(df_brazil[long&lat&cidadesNordeste]["LONG"], df_brazil[long&lat&cidadesNordeste]["LAT"], s=5, alpha=1, label='Nordeste', c ='#00BFFF')
plt.scatter(df_brazil[long&lat&cidadesCentroOeste]["LONG"], df_brazil[long&lat&cidadesCentroOeste]["LAT"], s=5, alpha=1, label='Centro Oeste', c ='#DB7093')
plt.scatter(df_brazil[long&lat&cidadesSul]["LONG"], df_brazil[long&lat&cidadesSul]["LAT"], s=5, alpha=1, label='SUL', c ='#9932CC')
plt.scatter(df_brazil[long&lat&cidadesSudeste]["LONG"], df_brazil[long&lat&cidadesSudeste]["LAT"], s=5, alpha=1, label='Sudeste', c ='#FF4500')
#plt.scatter(df_brazil[long&lat&~cidadesNordeste]["LONG"], df_brazil[long&lat&~cidadesNordeste]["LAT"], s=1, alpha=1, c = '#4b0082')
plt.legend()
plt.show()

In [ ]:
# Mapa do Brasil Capitais

long= df_brazil["LONG"] != 0
lat = df_brazil["LAT"] !=0 
cap = df_brazil['CAPITAL'] ==1
 
#Montando o Mapa
plt.figure(figsize=(10,10))
plt.title(" Cidades (Latitude e Longitude)")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.scatter(df_brazil[long&lat&cap]["LONG"], df_brazil[long&lat&cap]["LAT"], s=50, alpha=1, label='Capital city', c ='#ff4500')
plt.scatter(df_brazil[long&lat&~cap]["LONG"], df_brazil[long&lat&~cap]["LAT"], s=1, alpha=1, label='cidades', c = '#32cd32')
plt.legend()
plt.show()

### **Índice de Desenvolvimento Humano** 

In [ ]:
df_brazil.boxplot(column=['IDHM'])

# **Analisando as Regiões** 

## **Norte**

In [ ]:
DadosNorte = [{'Capitais':norte.loc[(norte['CAPITAL'] == 1)]['CITY'].count(),'Cidades': norte['CITY'].count(),'Habitantes': norte['IBGE_RES_POP'].sum(), 'Empresas': norte['COMP_TOT'].sum()}]
df_DadosNorte = pd.DataFrame(DadosNorte)
df_DadosNorte

In [ ]:
# Mapa do Norte

long= df_brazil["LONG"] !=0
lat = df_brazil["LAT"] !=0 
capNorte = norte['CAPITAL'] == 1

## use the scatter function
plt.figure(figsize=(10,10))
plt.title(" Cidades do Norte")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.scatter(df_brazil[long&lat&cidadesNorte]["LONG"], df_brazil[long&lat&cidadesNorte]["LAT"], s=5, alpha=1, label='Cidades', c ='#00BFFF')
plt.scatter(df_brazil[long&lat&capNorte]["LONG"], df_brazil[long&lat&capNorte]["LAT"], s=50, alpha=1, label='Capital city', c ='#ff4500')
plt.legend()
plt.show()

In [ ]:
norte.boxplot(column=['IDHM'])

In [ ]:
#Histograma do IDHM
histPE=norte['IDHM'].hist()

In [ ]:
#Comparando os IDHM dos estados do norte
# Criando lista
dados1 =  {'AC':norte[norte['STATE'] == 'AC'].IDHM_Educacao.mean() ,'AP':norte[norte['STATE'] == 'AP'].IDHM_Educacao.mean(), 'AM':norte[norte['STATE'] == 'AM'].IDHM_Educacao.mean(),'PA':norte[norte['STATE'] == 'PA'].IDHM_Educacao.mean(),'RO':norte[norte['STATE'] == 'RO'].IDHM_Educacao.mean(),'RR':norte[norte['STATE'] == 'RR'].IDHM_Educacao.mean(),'TO':norte[norte['STATE'] == 'TO'].IDHM_Educacao.mean()}
dados2 =  {'AC':norte[norte['STATE'] == 'AC'].IDHM.mean() ,'AP':norte[norte['STATE'] == 'AP'].IDHM.mean(), 'AM':norte[norte['STATE'] == 'AM'].IDHM.mean(),'PA':norte[norte['STATE'] == 'PI'].IDHM.mean(),'RO':norte[norte['STATE'] == 'RO'].IDHM.mean(),'RR':norte[norte['STATE'] == 'RR'].IDHM.mean(),'TO':norte[norte['STATE'] == 'TO'].IDHM.mean()}
dados3 =  {'AC':norte[norte['STATE'] == 'AC'].IDHM_Renda.mean() ,'AP':norte[norte['STATE'] == 'AP'].IDHM_Renda.mean(), 'AM':norte[norte['STATE'] == 'AM'].IDHM_Renda.mean(),'PA':norte[norte['STATE'] == 'PA'].IDHM_Renda.mean(),'RO':norte[norte['STATE'] == 'RO'].IDHM_Renda.mean(),'RR':norte[norte['STATE'] == 'RR'].IDHM_Renda.mean(),'TO':norte[norte['STATE'] == 'TO'].IDHM_Renda.mean()}
dados4 =  {'AC':norte[norte['STATE'] == 'AC'].IDHM_Longevidade.mean() ,'AP':norte[norte['STATE'] == 'AP'].IDHM_Longevidade.mean(), 'AM':norte[norte['STATE'] == 'AM'].IDHM_Longevidade.mean(),'PA':norte[norte['STATE'] == 'PA'].IDHM_Longevidade.mean(),'RO':norte[norte['STATE'] == 'RO'].IDHM_Longevidade.mean(),'RR':norte[norte['STATE'] == 'RR'].IDHM_Longevidade.mean(),'TO':norte[norte['STATE'] == 'TO'].IDHM_Longevidade.mean()}
Estados = list(dados1.keys())

MediaIDHM = list(dados2.values())
MediaIDHM_Educacao = list(dados1.values())
MediaIDHM_renda = list(dados3.values())
MediaIDHM_longevidade = list(dados4.values())

barWidth = 0.2

r1 = np.arange(len(MediaIDHM))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]

fig = plt.figure(figsize = (10, 5))

#Criando as barras
plt.bar(r1,MediaIDHM, color = '#00BFFF', width = barWidth, label = 'IDHM')
plt.bar(r2,MediaIDHM_Educacao, color = '#1049F0', width = barWidth, label = 'IDHM_Educacao')
plt.bar(r3,MediaIDHM_renda, color = '#D73E0D', width = barWidth, label = 'IDHM_renda')
plt.bar(r4,MediaIDHM_longevidade, color = '#0DD7A3', width = barWidth, label = 'IDHM_longevidade')


# criando Gráfico
plt.xlabel("Estados")
plt.xticks([r + barWidth for r in range(len(MediaIDHM))], Estados)
plt.ylabel("IDHM")
plt.title("Gráfico IDHM Médio norte")
plt.legend()
plt.show()

In [ ]:
#Capitais
norte.loc[norte['CAPITAL'] == 1][['CITY','STATE','IDHM','REGIAO_TUR','CATEGORIA_TUR']].sort_values('IDHM', ascending=False)

In [ ]:
#Melhores cidades turisticas do Norte
norte.loc[(norte['IDHM'] >= 0.650) & (norte['REGIAO_TUR'] != '0')][['CITY','STATE','IDHM','REGIAO_TUR','CATEGORIA_TUR', 'RURAL_URBAN']].sort_values('IDHM', ascending=False).head(10)

In [ ]:
# Criando Principais tipos de Industria no norte
dados =  {'Agriculture':norte.COMP_A.sum()/1000,'Indústrias extrativas':norte.COMP_B.sum()/1000,'Indústrias de transformação':norte.COMP_C.sum()/1000,'Eletricidade e gás':norte.COMP_D.sum()/1000,'Saneamento':norte.COMP_E.sum()/1000,'Construção':norte.COMP_F.sum()/1000,'Automobilística':norte.COMP_G.sum()/1000,'Transporte':norte.COMP_H.sum()/1000,'Hospedagem e alimentação':norte.COMP_I.sum()/1000,'Informação e comunicação':norte.COMP_J.sum()/1000,'Atividades financeiras':norte.COMP_K.sum()/1000,'Atividades imobiliárias':norte.COMP_L.sum()/1000,'Científicas e Técnicas':norte.COMP_M.sum()/1000,'atividades administrativas':norte.COMP_N.sum()/1000,'Administração e Segurança':norte.COMP_O.sum()/1000,'Educação':norte.COMP_P.sum()/1000,'Saúde':norte.COMP_Q.sum()/1000,'Cultura':norte.COMP_R.sum()/1000,'Outras':norte.COMP_S.sum()/1000,'Serviços domésticos':norte.COMP_T.sum()/1000,'Internacional':norte.COMP_U.sum()/1000}
Industria = list(dados.keys())
qtd = list(dados.values())
fig = plt.figure(figsize = (20, 10))

# criando Gráfico
plt.bar(Industria, qtd, color ='#00BFFF',width = 0.9)
plt.xticks(rotation = 90)
plt.xlabel("Áreas")
plt.ylabel("Quantidades(Mil)")
plt.title("Indústrias")
plt.show()

## **Nordeste**

In [ ]:
DadosNordeste = [{'Capitais':nordeste.loc[(nordeste['CAPITAL'] == 1)]['CITY'].count(),'Cidades': nordeste['CITY'].count(),'Habitantes': nordeste['IBGE_RES_POP'].sum(), 'Empresas': nordeste['COMP_TOT'].sum()}]
df_DadosNordeste = pd.DataFrame(DadosNordeste)
df_DadosNordeste

In [ ]:
# Mapa do Nordeste

long= df_brazil["LONG"] !=0
lat = df_brazil["LAT"] !=0 
capNordestete = nordeste['CAPITAL'] == 1

## use the scatter function
plt.figure(figsize=(10,10))
plt.title(" Cidades do Norte")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.scatter(df_brazil[long&lat&cidadesNordeste]["LONG"], df_brazil[long&lat&cidadesNordeste]["LAT"], s=5, alpha=1, label='Cidades', c ='#00BFFF')
plt.scatter(df_brazil[long&lat&capNordestete]["LONG"], df_brazil[long&lat&capNordestete]["LAT"], s=50, alpha=1, label='Capital city', c ='#ff4500')
plt.legend()
plt.show()

In [ ]:
nordeste.boxplot(column=['IDHM'])

In [ ]:
# Criando lista
dados1 =  {'PE':nordeste[nordeste['STATE'] == 'PE'].IDHM_Educacao.mean() ,'BA':nordeste[nordeste['STATE'] == 'BA'].IDHM_Educacao.mean(), 'MA':nordeste[nordeste['STATE'] == 'MA'].IDHM_Educacao.mean(),'PI':nordeste[nordeste['STATE'] == 'PI'].IDHM_Educacao.mean(),'CE':nordeste[nordeste['STATE'] == 'CE'].IDHM_Educacao.mean(),'RN':nordeste[nordeste['STATE'] == 'RN'].IDHM_Educacao.mean(),'PB':nordeste[nordeste['STATE'] == 'PB'].IDHM_Educacao.mean(),'AL':nordeste[nordeste['STATE'] == 'AL'].IDHM_Educacao.mean(),'SE':nordeste[nordeste['STATE'] == 'SE'].IDHM_Educacao.mean()}
dados2 =  {'PE':nordeste[nordeste['STATE'] == 'PE'].IDHM.mean() ,'BA':nordeste[nordeste['STATE'] == 'BA'].IDHM.mean(), 'MA':nordeste[nordeste['STATE'] == 'MA'].IDHM.mean(),'PI':nordeste[nordeste['STATE'] == 'PI'].IDHM.mean(),'CE':nordeste[nordeste['STATE'] == 'CE'].IDHM.mean(),'RN':nordeste[nordeste['STATE'] == 'RN'].IDHM.mean(),'PB':nordeste[nordeste['STATE'] == 'PB'].IDHM.mean(),'AL':nordeste[nordeste['STATE'] == 'AL'].IDHM.mean(),'SE':nordeste[nordeste['STATE'] == 'SE'].IDHM.mean()}
dados3 =  {'PE':nordeste[nordeste['STATE'] == 'PE'].IDHM_Renda.mean() ,'BA':nordeste[nordeste['STATE'] == 'BA'].IDHM_Renda.mean(), 'MA':nordeste[nordeste['STATE'] == 'MA'].IDHM_Renda.mean(),'PI':nordeste[nordeste['STATE'] == 'PI'].IDHM_Renda.mean(),'CE':nordeste[nordeste['STATE'] == 'CE'].IDHM_Renda.mean(),'RN':nordeste[nordeste['STATE'] == 'RN'].IDHM_Renda.mean(),'PB':nordeste[nordeste['STATE'] == 'PB'].IDHM_Renda.mean(),'AL':nordeste[nordeste['STATE'] == 'AL'].IDHM_Renda.mean(),'SE':nordeste[nordeste['STATE'] == 'SE'].IDHM_Renda.mean()}
dados4 =  {'PE':nordeste[nordeste['STATE'] == 'PE'].IDHM_Longevidade.mean() ,'BA':nordeste[nordeste['STATE'] == 'BA'].IDHM_Longevidade.mean(), 'MA':nordeste[nordeste['STATE'] == 'MA'].IDHM_Longevidade.mean(),'PI':nordeste[nordeste['STATE'] == 'PI'].IDHM_Longevidade.mean(),'CE':nordeste[nordeste['STATE'] == 'CE'].IDHM_Longevidade.mean(),'RN':nordeste[nordeste['STATE'] == 'RN'].IDHM_Longevidade.mean(),'PB':nordeste[nordeste['STATE'] == 'PB'].IDHM_Longevidade.mean(),'AL':nordeste[nordeste['STATE'] == 'AL'].IDHM_Longevidade.mean(),'SE':nordeste[nordeste['STATE'] == 'SE'].IDHM_Longevidade.mean()}
Estados = list(dados1.keys())

MediaIDHM = list(dados2.values())
MediaIDHM_Educacao = list(dados1.values())
MediaIDHM_renda = list(dados3.values())
MediaIDHM_longevidade = list(dados4.values())

barWidth = 0.2

r1 = np.arange(len(MediaIDHM))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]

fig = plt.figure(figsize = (10, 5))

#Criando as barras
plt.bar(r1,MediaIDHM, color = '#00BFFF', width = barWidth, label = 'IDHM')
plt.bar(r2,MediaIDHM_Educacao, color = '#1049F0', width = barWidth, label = 'IDHM_Educacao')
plt.bar(r3,MediaIDHM_renda, color = '#D73E0D', width = barWidth, label = 'IDHM_renda')
plt.bar(r4,MediaIDHM_longevidade, color = '#0DD7A3', width = barWidth, label = 'IDHM_longevidade')


# criando Gráfico
plt.xlabel("Estados")
plt.xticks([r + barWidth for r in range(len(MediaIDHM))], Estados)
plt.ylabel("IDHM")
plt.title("Gráfico IDHM Médio norte")
plt.legend()
plt.show()
plt.show()


In [ ]:
# Criando Principais tipos de Industria no nordeste
dados =  {'Agriculture':nordeste.COMP_A.sum()/1000,'Indústrias extrativas':nordeste.COMP_B.sum()/1000,'Indústrias de transformação':nordeste.COMP_C.sum()/1000,'Eletricidade e gás':nordeste.COMP_D.sum()/1000,'Saneamento':nordeste.COMP_E.sum()/1000,'Construção':nordeste.COMP_F.sum()/1000,'Automobilística':nordeste.COMP_G.sum()/1000,'Transporte':nordeste.COMP_H.sum()/1000,'Hospedagem e alimentação':nordeste.COMP_I.sum()/1000,'Informação e comunicação':nordeste.COMP_J.sum()/1000,'Atividades financeiras':nordeste.COMP_K.sum()/1000,'Atividades imobiliárias':nordeste.COMP_L.sum()/1000,'Científicas e Técnicas':nordeste.COMP_M.sum()/1000,'atividades administrativas':nordeste.COMP_N.sum()/1000,'Administração e Segurança':nordeste.COMP_O.sum()/1000,'Educação':nordeste.COMP_P.sum()/1000,'Saúde':nordeste.COMP_Q.sum()/1000,'Cultura':nordeste.COMP_R.sum()/1000,'Outras':nordeste.COMP_S.sum()/1000,'Serviços domésticos':nordeste.COMP_T.sum()/1000,'Internacional':nordeste.COMP_U.sum()/1000}
Industria = list(dados.keys())
qtd = list(dados.values())
fig = plt.figure(figsize = (20, 10))

# criando Gráfico
plt.bar(Industria, qtd, color ='#00BFFF',width = 0.9)
plt.xticks(rotation = 90)
plt.xlabel("Áreas")
plt.ylabel("Quantidades(Mil)")
plt.title("Indústrias")
plt.show()

# **Sul**

In [ ]:
DadosSul = [{'Capitais':sul.loc[(sul['CAPITAL'] == 1)]['CITY'].count(),'Cidades': sul['CITY'].count(),'Habitantes': sul['IBGE_RES_POP'].sum(), 'Empresas': sul['COMP_TOT'].sum()}]
df_DadosSul = pd.DataFrame(DadosSul)
df_DadosSul

In [ ]:
# Mapa do Sul

long= df_brazil["LONG"] !=0
lat = df_brazil["LAT"] !=0 
capSul = sul['CAPITAL'] == 1

## use the scatter function
plt.figure(figsize=(10,10))
plt.title(" Cidades do Norte")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.scatter(df_brazil[long&lat&cidadesSul]["LONG"], df_brazil[long&lat&cidadesSul]["LAT"], s=5, alpha=1, label='Cidades', c ='#00BFFF')
plt.scatter(df_brazil[long&lat&capSul]["LONG"], df_brazil[long&lat&capSul]["LAT"], s=50, alpha=1, label='Capital city', c ='#ff4500')
plt.legend()
plt.show()

In [ ]:
sul.boxplot(column=['IDHM'])

In [ ]:
#Histograma do IDHM
histSUL=sul['IDHM'].hist()

In [ ]:
# Criando lista
dados1 =  {'PR':sul[sul['STATE'] == 'PR'].IDHM_Educacao.mean() ,'RS':sul[sul['STATE'] == 'RS'].IDHM_Educacao.mean(), 'SC':sul[sul['STATE'] == 'SC'].IDHM_Educacao.mean()}
dados2 =  {'PR':sul[sul['STATE'] == 'PR'].IDHM.mean() ,'RS':sul[sul['STATE'] == 'RS'].IDHM.mean(), 'SC':sul[sul['STATE'] == 'SC'].IDHM.mean()}
dados3 =  {'PR':sul[sul['STATE'] == 'PR'].IDHM_Renda.mean() ,'RS':sul[sul['STATE'] == 'RS'].IDHM_Renda.mean(), 'SC':sul[sul['STATE'] == 'SC'].IDHM_Renda.mean()}
dados4 =  {'PR':sul[sul['STATE'] == 'PR'].IDHM_Longevidade.mean() ,'RS':sul[sul['STATE'] == 'RS'].IDHM_Longevidade.mean(), 'SC':sul[sul['STATE'] == 'SC'].IDHM_Longevidade.mean()}
Estados = list(dados1.keys())

MediaIDHM = list(dados2.values())
MediaIDHM_Educacao = list(dados1.values())
MediaIDHM_renda = list(dados3.values())
MediaIDHM_longevidade = list(dados4.values())

barWidth = 0.2

r1 = np.arange(len(MediaIDHM))
r2 = [x + barWidth for x in r1]
r3 = [x + barWidth for x in r2]
r4 = [x + barWidth for x in r3]

fig = plt.figure(figsize = (10, 5))

#Criando as barras
plt.bar(r1,MediaIDHM, color = '#00BFFF', width = barWidth, label = 'IDHM')
plt.bar(r2,MediaIDHM_Educacao, color = '#1049F0', width = barWidth, label = 'IDHM_Educacao')
plt.bar(r3,MediaIDHM_renda, color = '#D73E0D', width = barWidth, label = 'IDHM_renda')
plt.bar(r4,MediaIDHM_longevidade, color = '#0DD7A3', width = barWidth, label = 'IDHM_longevidade')


# criando Gráfico
plt.xlabel("Estados")
plt.xticks([r + barWidth for r in range(len(MediaIDHM))], Estados)
plt.ylabel("IDHM")
plt.title("Gráfico IDHM Médio norte")
plt.legend()
plt.show()


In [ ]:
# Criando Principais tipos de Industria no sul
dados =  {'Agriculture':sul.COMP_A.sum()/1000,'Indústrias extrativas':sul.COMP_B.sum()/1000,'Indústrias de transformação':sul.COMP_C.sum()/1000,'Eletricidade e gás':sul.COMP_D.sum()/1000,'Saneamento':sul.COMP_E.sum()/1000,'Construção':sul.COMP_F.sum()/1000,'Automobilística':sul.COMP_G.sum()/1000,'Transporte':sul.COMP_H.sum()/1000,'Hospedagem e alimentação':sul.COMP_I.sum()/1000,'Informação e comunicação':sul.COMP_J.sum()/1000,'Atividades financeiras':sul.COMP_K.sum()/1000,'Atividades imobiliárias':sul.COMP_L.sum()/1000,'Científicas e Técnicas':sul.COMP_M.sum()/1000,'atividades administrativas':sul.COMP_N.sum()/1000,'Administração e Segurança':sul.COMP_O.sum()/1000,'Educação':sul.COMP_P.sum()/1000,'Saúde':sul.COMP_Q.sum()/1000,'Cultura':sul.COMP_R.sum()/1000,'Outras':sul.COMP_S.sum()/1000,'Serviços domésticos':sul.COMP_T.sum()/1000,'Internacional':sul.COMP_U.sum()/1000}
Industria = list(dados.keys())
qtd = list(dados.values())
fig = plt.figure(figsize = (20, 10))

# criando Gráfico
plt.bar(Industria, qtd, color ='#00BFFF',width = 0.9)
plt.xticks(rotation = 90)
plt.xlabel("Áreas")
plt.ylabel("Quantidades(Mil)")
plt.title("Indústrias")
plt.show()

In [ ]:
nordeste.loc[(nordeste['IDHM'] <= 0.5) & (nordeste['STATE'] == "PE")][['CITY','STATE','IDHM']]

In [ ]:
#As 10 cidades com IDHM maior 
nordeste.loc[(nordeste['IDHM'] >= 0.650) & (nordeste['STATE'] == "PE")][['CITY','STATE','IDHM','IBGE_RES_POP','GDP','COMP_TOT']].sort_values('IDHM', ascending=False).head(10)

In [ ]:
# Criando lista
dados =  {'Agriculture':nordeste[nordeste['STATE'] == 'PE'].COMP_A.sum()/1000,'Indústrias extrativas':nordeste[nordeste['STATE'] == 'PE'].COMP_B.sum()/1000,'Indústrias de transformação':nordeste[nordeste['STATE'] == 'PE'].COMP_C.sum()/1000,'Eletricidade e gás':nordeste[nordeste['STATE'] == 'PE'].COMP_D.sum()/1000,'Saneamento':nordeste[nordeste['STATE'] == 'PE'].COMP_E.sum()/1000,'Construção':nordeste[nordeste['STATE'] == 'PE'].COMP_F.sum()/1000,'Automobilística':nordeste[nordeste['STATE'] == 'PE'].COMP_G.sum()/1000,'Transporte':nordeste[nordeste['STATE'] == 'PE'].COMP_H.sum()/1000,'Hospedagem e alimentação':nordeste[nordeste['STATE'] == 'PE'].COMP_I.sum()/1000,'Informação e comunicação':nordeste[nordeste['STATE'] == 'PE'].COMP_J.sum()/1000,'Atividades financeiras':nordeste[nordeste['STATE'] == 'PE'].COMP_K.sum()/1000,'Atividades imobiliárias':nordeste[nordeste['STATE'] == 'PE'].COMP_L.sum()/1000,'Científicas e Técnicas':nordeste[nordeste['STATE'] == 'PE'].COMP_M.sum()/1000,'atividades administrativas':nordeste[nordeste['STATE'] == 'PE'].COMP_N.sum()/1000,'Administração e Segurança':nordeste[nordeste['STATE'] == 'PE'].COMP_O.sum()/1000,'Educação':nordeste[nordeste['STATE'] == 'PE'].COMP_P.sum()/1000,'Saúde':nordeste[nordeste['STATE'] == 'PE'].COMP_Q.sum()/1000,'Cultura':nordeste[nordeste['STATE'] == 'PE'].COMP_R.sum()/1000,'Outras':nordeste[nordeste['STATE'] == 'PE'].COMP_S.sum()/1000,'Serviços domésticos':nordeste[nordeste['STATE'] == 'PE'].COMP_T.sum()/1000,'Internacional':nordeste[nordeste['STATE'] == 'PE'].COMP_U.sum()/1000}
Industria = list(dados.keys())
qtd = list(dados.values())
fig = plt.figure(figsize = (20, 10))

# criando Gráfico
plt.bar(Industria, qtd, color ='#00BFFF',width = 0.9)
plt.xticks(rotation = 50)
plt.xlabel("Áreas")
plt.ylabel("Quantidades")
plt.title("Indústrias")
plt.show()

# **----------------------------------------------------------------------------** 

# **Leandro aqui**

In [ ]:
df_brazil.describe()

In [ ]:
#  Eliminando linhas duplicadas, Cidades: Assis, Paraty e Nova Fátima
df_brazil = df_brazil.drop_duplicates()

In [ ]:
# Cidade de Paraty possui dois dados sobre a agência de correios, eliminei o diferente de 1 (pesquisado no google que só existe uma agência)
df_brazil.drop(df_brazil.loc[(df_brazil['CITY']=='Paraty') & (df_brazil['POST_OFFICES'] != 1)].index, inplace=True)

In [ ]:
# gráfico mostrando que só há uber na área urbana
df_brazil.groupby('RURAL_URBAN')['UBER'].sum().plot(kind='bar',figsize=(5,5))

In [ ]:
#Porcentagem correpondente de cada tipologia

df_brazil.groupby(['RURAL_URBAN']).size().plot(kind='pie', autopct='%0.2f%%', figsize=(10,10))

In [ ]:
# De acordo com o IBGE a menor cidade do Brasil tem 3565 km², dando o código abaixo , retona 3 cidades com área 0
df_brazil[(df_brazil['AREA'] < 3.565)]
df_brazil.drop(df_brazil.loc[(df_brazil['AREA'] <  3565)].index, inplace=True)

In [ ]:
# De acordo com o IBGE a  cidade do Brasil menos povoada tem 776 pessoas
df_brazil[(df_brazil['IBGE_RES_POP'] < 776)]
df_brazil.drop(df_brazil.loc[(df_brazil['IBGE_RES_POP']< 3.565)].index, inplace=True)

<h1> <font color ="yellow">Limpeza</font> <font color="green">dos dados</font><h1>

<h3>Univariado usando Z-Score robusto<h3>

In [ ]:
df_brazil['TAXES'].value_counts()

In [ ]:
df_brazil['TAXES'].describe()

In [ ]:
# Plotando o gráfico para ver as outliers
df_brazil['TAXES'].plot.box()

In [ ]:
df_brazil['TAXES'].hist()

In [ ]:
 # Como visto em cima , os dados não estão simétricos , então aplicamos log10
 df_brazil['TAXES_Log'] = log10(df_brazil['TAXES'])
 df_brazil['TAXES_Log'].hist()

In [ ]:
# Usando z-core robusto
from numpy import abs
mad = abs(df_brazil['TAXES_Log'] - df_brazil['TAXES_Log'].median()).median()*(1/0.6745)
print(mad)

In [ ]:
# Agora vamos plotar o z-score robusto
(abs(df_brazil['TAXES_Log']-df_brazil['TAXES_Log'].median())/mad).hist()

In [ ]:
# Usando a limiar > 2.5, encontro os outliers
df_brazil[abs(df_brazil['TAXES_Log']-df_brazil['TAXES_Log'].median())/mad > 2.5]

In [ ]:
# Gerando novo dataframe, eliminando os outros
df_brazil = df_brazil[abs(df_brazil['TAXES_Log']-df_brazil['TAXES_Log'].median())/mad < 2.5]

In [ ]:
df_brazil['TAXES'].plot.box()